In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from torchvision import models
from tqdm import tqdm
from collections import Counter
import wandb


In [2]:
data_dir_train = "/kaggle/input/skin-disease-dataset/Skin_Disease_Dataset/Train"
data_dir_test = "/kaggle/input/skin-disease-dataset/Skin_Disease_Dataset/Test"
data_dir_val = "/kaggle/input/skin-disease-dataset/Skin_Disease_Dataset/Val"


In [3]:
# data_dir_train = "/kaggle/input/vegetable-image-dataset/Vegetable Images/test"
# data_dir_test = "/kaggle/input/vegetable-image-dataset/Vegetable Images/train"

In [4]:

# Data transformations
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Datasets
train_dataset = datasets.ImageFolder(root=data_dir_train, transform=data_transforms["train"])
val_dataset = datasets.ImageFolder(root=data_dir_val, transform=data_transforms["val"])
test_dataset = datasets.ImageFolder(root=data_dir_test, transform=data_transforms["test"])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Class information
class_names = train_dataset.classes
num_classes = len(class_names)

print(f"Classes: {class_names}")
print(f"Number of classes: {num_classes}")
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")


Classes: ['1. Eczema', '10. Warts Molluscum and other Viral Infections', '2. Melanoma', '3. Atopic Dermatitis', '4. Basal Cell Carcinoma (BCC)', '5. Melanocytic Nevi (NV)', '6. Benign Keratosis-like Lesions (BKL)', '7. Psoriasis pictures Lichen Planus and related diseases', '8. Seborrheic Keratoses and other Benign Tumors', '9. Tinea Ringworm Candidiasis and other Fungal Infections']
Number of classes: 10
Number of training samples: 19003
Number of validation samples: 2711
Number of test samples: 5439


In [5]:
print(class_names)
num_classes

['1. Eczema', '10. Warts Molluscum and other Viral Infections', '2. Melanoma', '3. Atopic Dermatitis', '4. Basal Cell Carcinoma (BCC)', '5. Melanocytic Nevi (NV)', '6. Benign Keratosis-like Lesions (BKL)', '7. Psoriasis pictures Lichen Planus and related diseases', '8. Seborrheic Keratoses and other Benign Tumors', '9. Tinea Ringworm Candidiasis and other Fungal Infections']


10

In [6]:
from torch.utils.data import WeightedRandomSampler

# Tính trọng số mẫu
class_sample_counts = Counter([label for _, label in train_dataset])
weights = 1. / np.array([class_sample_counts[i] for i in range(num_classes)])
sample_weights = [weights[label] for _, label in train_dataset]

# Sampler
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)
train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler, num_workers=2)


In [7]:
# Tính trọng số cho các lớp
y_train = [label for _, labels in train_loader for label in labels.numpy()]
class_weights = compute_class_weight(class_weight="balanced", classes=np.arange(num_classes), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to("cuda")

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
# Load pre-trained VGG16
from torchvision import models
import torch.nn as nn

model = models.vgg16(pretrained=True)

# Freeze tất cả các lớp pre-trained
for param in model.features.parameters():
    param.requires_grad = False

# Tùy chỉnh classifier cho bài toán hiện tại
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, num_classes)

# Di chuyển mô hình sang thiết bị
model = model.to(device)

# Đặt tốc độ học riêng cho từng nhóm tham số
optimizer = torch.optim.Adam([
    {'params': model.features.parameters(), 'lr': 1e-4},  # Tốc độ học nhỏ hơn cho các lớp pre-trained
    {'params': model.classifier.parameters(), 'lr': 1e-3}  # Tốc độ học lớn hơn cho các lớp fine-tuned
])


In [22]:
import os

# Tạo thư mục nếu chưa tồn tại
save_dir = "/kaggle/working/check"
os.makedirs(save_dir, exist_ok=True)

In [19]:
import wandb
import timm

# Khởi tạo Wandb
wandb.init(
    project="skin-disease-classification",
    id="VGG-16-skin-fine-tune hoang anh",
    config={
        "epochs": 100,
        "batch_size": 64,
        "learning_rate": 0.001,
        "optimizer": "Adam",
        "model": "VGG16",
        "num_classes": num_classes
    }
)


# # Lấy config từ Wandb
# config = wandb.config

# Thiết lập thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Khởi tạo mô hình và các thành phần
# model = model_powerup.to(device)
# # model = model_from_scratch_2
# criterion = nn.CrossEntropyLoss(weight=class_weights)
# optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
# Tạo dataloaders dictionary
dataloaders = {
    "train": train_loader,
    "val": val_loader,
    "test": test_loader,
}


# checkpoint = torch.load("/kaggle/working/check/checkpoint4.pth", map_location=device)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# start_epoch = checkpoint['epoch']


In [24]:
# Sử dụng CrossEntropy với trọng số lớp
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=wandb.config.learning_rate)

# Sử dụng Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Scheduler để giảm learning rate khi loss không giảm
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [25]:
def train_and_evaluate_with_wandb(model, dataloaders, criterion, optimizer, device, num_epochs):
    train_loss_history, train_acc_history = [], []
    val_loss_history, val_acc_history = [], []
    best_val_acc = 0.0  # Biến để lưu trữ giá trị val accuracy tốt nhất

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        wandb.log({"epoch": epoch + 1})
        
        # Pha train được thực hiện mọi epoch
        model.train()
        running_loss, running_corrects = 0.0, 0

        for batch_idx, (inputs, labels) in enumerate(dataloaders['train']):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                loss.backward()
                optimizer.step()

                #Tính Grad Norm
                total_norm = 0
                for p in model.parameters():
                    if p.grad is not None:
                        total_norm += p.grad.norm().item() ** 2
                total_norm = total_norm ** 0.5
                wandb.log({"grad_norm": total_norm})

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Log hình ảnh từ batch đầu tiên
            if batch_idx == 0:
                images = wandb.Image(inputs[:8].cpu(), caption="Ground Truth: {}".format(labels[:8].cpu()))
                wandb.log({"samples": images})

        # Tính toán loss và accuracy sau pha train
        train_loss = running_loss / len(dataloaders['train'].dataset)
        train_acc = running_corrects.double() / len(dataloaders['train'].dataset)

        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.item())
        wandb.log({
            "train_loss": train_loss,
            "train_accuracy": train_acc.item()
        })

        print(f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f}")

        # Pha validation chỉ chạy mỗi 5 epoch
        if epoch % 5 == 4:
            model.eval()
            running_loss, running_corrects = 0.0, 0

            for batch_idx, (inputs, labels) in enumerate(dataloaders['val']):
                inputs, labels = inputs.to(device), labels.to(device)

                with torch.set_grad_enabled(False):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

            # Tính toán loss và accuracy sau pha validation
            val_loss = running_loss / len(dataloaders['val'].dataset)
            val_acc = running_corrects.double() / len(dataloaders['val'].dataset)

            val_loss_history.append(val_loss)
            val_acc_history.append(val_acc.item())
            wandb.log({
                "val_loss": val_loss,
                "val_accuracy": val_acc.item()
            })

            print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

            # Kiểm tra và lưu mô hình nếu val accuracy cao nhất
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                model_path = f"/kaggle/working/best_model_vit_16_tiny_custom_epoch_{epoch + 1}.pth"  # Tên file có index của epoch
                torch.save(model.state_dict(), model_path)
                wandb.save(model_path)
                print(f"Best model saved at epoch {epoch + 1} with val accuracy {best_val_acc:.4f}")

    # Lưu mô hình cuối cùng vào Wandb
    torch.save(model.state_dict(), "fine_tuned_model.pth")
    wandb.save("fine_tune_model.pth")
    print("Final model saved.")

    return train_loss_history, train_acc_history, val_loss_history, val_acc_history

In [27]:
# Huấn luyện và đánh giá mô hình
train_loss_history, train_acc_history, val_loss_history, val_acc_history = train_and_evaluate_with_wandb(
    model=model,
    dataloaders=dataloaders,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=100
)
from sklearn.metrics import classification_report, accuracy_score, average_precision_score, confusion_matrix
import pandas as pd

def evaluate_on_test(model, test_loader, device, class_names):
    model.eval()  # Chuyển sang chế độ đánh giá
    all_preds = []
    all_labels = []
    all_probs = []

    print("Evaluating on test dataset...")
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(probs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # Tính các chỉ số tổng thể
    accuracy = accuracy_score(all_labels, all_preds)
    report = classification_report(
        all_labels, 
        all_preds, 
        target_names=class_names, 
        digits=4, 
        output_dict=True
    )
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1 = report['weighted avg']['f1-score']

    # mAP (Mean Average Precision)
    binarized_labels = pd.get_dummies(all_labels).to_numpy()
    mAP = average_precision_score(binarized_labels, all_probs, average='macro')

    # Log các chỉ số tổng thể lên WandB
    # wandb.log({
    #     "test_accuracy": accuracy,
    #     "test_precision_weighted": precision,
    #     "test_recall_weighted": recall,
    #     "test_f1_weighted": f1,
    #     "test_mAP": mAP
    # })

    # Log các chỉ số theo từng lớp
    # for class_name, metrics in report.items():
    #     if class_name in class_names:  # Chỉ log cho các lớp, bỏ qua "accuracy", "weighted avg", "macro avg"
            # wandb.log({
            #     f"test_precision_{class_name}": metrics['precision'],
            #     f"test_recall_{class_name}": metrics['recall'],
            #     f"test_f1_{class_name}": metrics['f1-score']
            # })

    # In các chỉ số tổng thể
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (weighted): {precision:.4f}, Recall (weighted): {recall:.4f}, F1-score (weighted): {f1:.4f}, mAP: {mAP:.4f}")

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
    print("Confusion Matrix:")
    print(cm_df)
    # wandb.log({"test_confusion_matrix": wandb.Table(dataframe=cm_df)})

    # Hiển thị một số mẫu đúng và sai
    print("\nSample Predictions:")
    correct_samples = [(i, p) for i, p in enumerate(zip(all_labels, all_preds)) if p[0] == p[1]]
    incorrect_samples = [(i, p) for i, p in enumerate(zip(all_labels, all_preds)) if p[0] != p[1]]

    print("Correct Predictions (first 5):")
    for idx, (true, pred) in correct_samples[:5]:
        print(f"Sample {idx}: True = {class_names[true]}, Pred = {class_names[pred]}")

    print("Incorrect Predictions (first 5):")
    for idx, (true, pred) in incorrect_samples[:5]:
        print(f"Sample {idx}: True = {class_names[true]}, Pred = {class_names[pred]}")

    return {
        "accuracy": accuracy,
        "precision_weighted": precision,
        "recall_weighted": recall,
        "f1_score_weighted": f1,
        "mAP": mAP,
        "class_metrics": report
    }


# Gọi evaluate_on_test sau khi huấn luyện
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)
dataloaders["test"] = test_dataloader



# Đánh giá trên test set
test_metrics = evaluate_on_test(model, dataloaders["test"], device, class_names)
print(f"Test Set Metrics:\n{test_metrics}")
# wandb.log({"test_metrics": test_metrics})


Epoch 1/100
Train Loss: 1.0248 Acc: 0.6046
Epoch 2/100
Train Loss: 0.9602 Acc: 0.6312
Epoch 3/100
Train Loss: 0.9180 Acc: 0.6527
Epoch 4/100
Train Loss: 0.8827 Acc: 0.6633
Epoch 5/100
Train Loss: 0.8513 Acc: 0.6764
Val Loss: 0.8528 Acc: 0.6736


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Best model saved at epoch 5 with val accuracy 0.6736
Epoch 6/100
Train Loss: 0.8208 Acc: 0.6926
Epoch 7/100
Train Loss: 0.7890 Acc: 0.6987
Epoch 8/100
Train Loss: 0.7678 Acc: 0.7113
Epoch 9/100
Train Loss: 0.7421 Acc: 0.7227
Epoch 10/100
Train Loss: 0.7336 Acc: 0.7233
Val Loss: 0.8287 Acc: 0.6842
Best model saved at epoch 10 with val accuracy 0.6842
Epoch 11/100
Train Loss: 0.7066 Acc: 0.7335
Epoch 12/100
Train Loss: 0.6908 Acc: 0.7418
Epoch 13/100
Train Loss: 0.6774 Acc: 0.7450
Epoch 14/100
Train Loss: 0.6453 Acc: 0.7588
Epoch 15/100
Train Loss: 0.6451 Acc: 0.7584
Val Loss: 0.7749 Acc: 0.7245
Best model saved at epoch 15 with val accuracy 0.7245
Epoch 16/100
Train Loss: 0.6198 Acc: 0.7683
Epoch 17/100
Train Loss: 0.6146 Acc: 0.7696
Epoch 18/100
Train Loss: 0.6065 Acc: 0.7722
Epoch 19/100
Train Loss: 0.5870 Acc: 0.7843
Epoch 20/100
Train Loss: 0.5812 Acc: 0.7869
Val Loss: 0.7660 Acc: 0.7311
Best model saved at epoch 20 with val accuracy 0.7311
Epoch 21/100
Train Loss: 0.5692 Acc: 0.787

In [ ]:
# # Tải trọng số đã lưu vào mô hình
# model_path = "/kaggle/working/check/fine_tuned_model.pth"

# # Hàm tải trọng số
# try:
#     model_powerup.load_state_dict(torch.load(model_path, map_location=device))
#     model_powerup.eval()  # Đặt mô hình vào chế độ đánh giá
#     print("Trọng số đã được tải thành công.")
# except Exception as e:
#     print(f"Không thể tải trọng số: {e}")
